In [ ]:
import os
from operator import itemgetter
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, Dropout, Masking

from keras.utils import to_categorical

**Предсказание времени отказа для ракетных двигателей.**


Скачаем данные с сайта НАСА https://ti.arc.nasa.gov/c/6/ .  Используем FD_001

In [ ]:
train_sample = pd.read_csv('train_FD001.txt', sep=" ", header=None)
test_sample = pd.read_csv('test_FD001.txt', sep=" ", header=None)
train_sample.head(5)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


In [ ]:
train_sample.shape[0]

20631

In [ ]:
test_sample.shape[0]

13096

Сгруппируем датафреймы по колонке 0 (типу двигателя)

In [ ]:
train_seq = []
test_seq = []
for typ in train_sample[0].unique():
  train_seq.append(train_sample[train_sample[:][0] == typ].reset_index(drop=True))
  test_seq.append(test_sample[test_sample[:][0] == typ].reset_index(drop=True))
train_seq[2].head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,3,1,0.0008,0.0005,100.0,518.67,642.04,1584.20,1398.13,14.62,...,8138.40,8.4207,0.03,391,2388,100.0,38.96,23.3205,NaN,NaN
1,3,2,-0.0010,0.0000,100.0,518.67,642.66,1587.04,1398.62,14.62,...,8137.38,8.3949,0.03,390,2388,100.0,39.07,23.4369,NaN,NaN
2,3,3,0.0013,-0.0002,100.0,518.67,642.07,1580.75,1401.10,14.62,...,8137.20,8.3820,0.03,393,2388,100.0,39.03,23.3162,NaN,NaN
3,3,4,0.0008,-0.0002,100.0,518.67,642.50,1580.12,1395.76,14.62,...,8139.35,8.3940,0.03,391,2388,100.0,38.94,23.4901,NaN,NaN
4,3,5,0.0020,0.0004,100.0,518.67,641.97,1581.48,1394.05,14.62,...,8135.99,8.4233,0.03,391,2388,100.0,38.90,23.4190,NaN,NaN


Добавим RUL = max по всем тактам

In [ ]:
# Только для train: RUL = max_cycle - current_cycle
for seq in train_seq:
    max_cycle = seq[1].max()
    seq["RUL"] = max_cycle - seq[1]

Создадим колонку y - эталлонные данные (через сколько тактов двигатель выйдет из строя). Удалим константные колнки

In [ ]:

for seq in train_seq:
    seq["y"] = seq["RUL"]
    seq.drop(columns=[0, 1, 26, 27, "RUL"], inplace=True)

for seq in test_seq:
    seq.drop(columns=[0, 1, 26, 27], inplace=True)

Нормализуем данные. Сделаем из датафрейма последовательности размера <=50. В качестве labels возьмем данные колонки "y" после соответствующих отсчетов

In [ ]:
from sklearn.preprocessing import StandardScaler


def normalize_data(train_seq):
    all_features = np.vstack([seq.iloc[:, :24].values for seq in train_seq])
    scaler = StandardScaler().fit(all_features)

    normalized_seq = []
    for seq in train_seq:
        seq_norm = seq.copy()
        seq_norm.iloc[:, :24] = scaler.transform(seq.iloc[:, :24])
        normalized_seq.append(seq_norm)
    return normalized_seq, scaler

train_seq_norm, scaler = normalize_data(train_seq)

def make_train_samples(seq, sequence_length=50):
    X, y = [], []
    for i in range(sequence_length, len(seq)):
        X.append(seq.iloc[i-sequence_length:i, :24].values)
        y.append(seq.iloc[i, 24])
    return np.array(X), np.array(y)

X_all, y_all = [], []
for seq in train_seq_norm:
    X_tmp, y_tmp = make_train_samples(seq, sequence_length=50)
    X_all.extend(X_tmp)
    y_all.extend(y_tmp)

train_X = np.array(X_all)
train_y = np.array(y_all)

train_y_normalized = train_y / train_y.max()


Запустим обучение, предусмотрев метод ранней остановки, и batch normalization

In [ ]:
from keras.callbacks import EarlyStopping

model = Sequential([
    Masking(mask_value=0., input_shape=(None, 24)),
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3),
    BatchNormalization(),
    LSTM(64, return_sequences=False, dropout=0.4, recurrent_dropout=0.4),
    BatchNormalization(),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_X, train_y_normalized,
    epochs=40,
    batch_size=128,
    validation_split=0.2,
    shuffle=False,
    callbacks=[callback],
    verbose=2
)

Epoch 1/40
98/98 - 48s - 485ms/step - loss: 0.6074 - mae: 0.5873 - val_loss: 0.0337 - val_mae: 0.1272
Epoch 2/40
98/98 - 38s - 390ms/step - loss: 0.1661 - mae: 0.3155 - val_loss: 0.0195 - val_mae: 0.1065
Epoch 3/40
98/98 - 40s - 413ms/step - loss: 0.0795 - mae: 0.2144 - val_loss: 0.0208 - val_mae: 0.1083
Epoch 4/40
98/98 - 41s - 420ms/step - loss: 0.0501 - mae: 0.1670 - val_loss: 0.0204 - val_mae: 0.1062
Epoch 5/40
98/98 - 41s - 414ms/step - loss: 0.0383 - mae: 0.1450 - val_loss: 0.0210 - val_mae: 0.1103
Epoch 6/40
98/98 - 35s - 357ms/step - loss: 0.0316 - mae: 0.1310 - val_loss: 0.0198 - val_mae: 0.1043
Epoch 7/40
98/98 - 35s - 353ms/step - loss: 0.0278 - mae: 0.1231 - val_loss: 0.0197 - val_mae: 0.1079


In [ ]:
# Нормализуем test_seq с помощью scaler из train
rul_true = pd.read_csv('RUL_FD001.txt', header=None)
test_seq_norm = []
for seq in test_seq:
    seq_norm = seq.copy()
    seq_norm.iloc[:, :24] = scaler.transform(seq.iloc[:, :24])
    test_seq_norm.append(seq_norm)

# Создаём последовательности: последние 50 шагов каждого двигателя
SEQUENCE_LENGTH = 50
X_test = []

for seq in test_seq_norm:
    if len(seq) >= SEQUENCE_LENGTH:
        X_test.append(seq.iloc[-SEQUENCE_LENGTH:, :24].values)
    else:
        # Паддинг нулями слева
        pad = SEQUENCE_LENGTH - len(seq)
        padded = np.pad(seq.iloc[:, :24].values, ((pad, 0), (0, 0)), 'constant')
        X_test.append(padded)

X_test = np.array(X_test)


y_pred_norm = modelr.predict(X_test).flatten()
y_pred = y_pred_norm * train_y.max()  # Обратно в циклы

# Истинные RUL
y_true = rul_true.values.flatten()

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print(f"RMSE:  {rmse:.2f}")
print(f"MAE:   {mae:.2f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 330ms/step
RMSE:  28.86
MAE:   20.95


Предусмотрим сортировку по длинам последовательностей

In [ ]:
length_x = [len(x) for x in train_X]
args = np.argsort(length_x)

sort_X = np.empty_like(train_X)
sort_y = np.empty_like(train_y)

for i, arg in enumerate(args):
  sort_X[i] = train_X[arg]
  sort_y[i] = train_y[arg]


In [ ]:
lengths = [len(seq) for seq in train_seq_norm]
sorted_indices = np.argsort(lengths)[::-1]
sorted_seqs = [train_seq_norm[i] for i in sorted_indices]

In [ ]:
SEQUENCE_LENGTH = 50
X_list, y_list = [], []

for seq in sorted_seqs:
    if len(seq) > SEQUENCE_LENGTH:
        for i in range(SEQUENCE_LENGTH, len(seq)):
            X_list.append(seq.iloc[i-SEQUENCE_LENGTH:i, :24].values)
            y_list.append(seq.iloc[i, 24])


sort_X_norm = np.array(X_list)
train_y = np.array(y_list)

In [ ]:
rul_max = train_y.max()
sort_y_norm = train_y / rul_max

In [ ]:
modelr.compile( loss='mean_squared_error',
optimizer='adam',
metrics =['mae'])

history = modelr.fit(sort_X_norm, sort_y_norm,
epochs=maxEpochs,
batch_size=miniBatchSize,
validation_split=0.2,
verbose=2, callbacks=[callback], shuffle=False)

Epoch 1/40
98/98 - 27s - 273ms/step - loss: 15.5334 - mae: 3.8699 - val_loss: 8.0870 - val_mae: 2.8378
Epoch 2/40
98/98 - 16s - 162ms/step - loss: 5.4270 - mae: 2.2912 - val_loss: 3.3408 - val_mae: 1.8238
Epoch 3/40
98/98 - 21s - 218ms/step - loss: 2.1143 - mae: 1.4258 - val_loss: 1.4638 - val_mae: 1.2060
Epoch 4/40
98/98 - 20s - 205ms/step - loss: 0.8851 - mae: 0.9143 - val_loss: 0.7261 - val_mae: 0.8484
Epoch 5/40
98/98 - 17s - 175ms/step - loss: 0.3892 - mae: 0.5948 - val_loss: 0.3639 - val_mae: 0.5985
Epoch 6/40
98/98 - 15s - 157ms/step - loss: 0.1771 - mae: 0.3900 - val_loss: 0.2051 - val_mae: 0.4467
Epoch 7/40
98/98 - 24s - 241ms/step - loss: 0.0873 - mae: 0.2631 - val_loss: 0.0950 - val_mae: 0.3008
Epoch 8/40
98/98 - 22s - 220ms/step - loss: 0.0484 - mae: 0.1880 - val_loss: 0.0565 - val_mae: 0.2271
Epoch 9/40
98/98 - 23s - 230ms/step - loss: 0.0334 - mae: 0.1488 - val_loss: 0.0333 - val_mae: 0.1700
Epoch 10/40
98/98 - 36s - 363ms/step - loss: 0.0276 - mae: 0.1292 - val_loss: 0.0

In [ ]:
y_pred_norm = modelr.predict(X_test).flatten()
y_pred = y_pred_norm * train_y.max()  # Обратно в циклы

# Истинные RUL
y_true = rul_true.values.flatten()

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print(f"RMSE:  {rmse:.2f}")
print(f"MAE:   {mae:.2f}")


1/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 305ms/step
RMSE:  23.34
MAE:   18.34
